In [110]:
!pip install torch


In [111]:
import pandas as pd

In [328]:
dataset = pd.read_csv('most_popular_repositories_1000.csv')

dataset['Description'] = dataset['Description'].fillna('')
dataset['description_length'] = dataset['Description'].apply(len)

columns_to_drop = ['Name', 'FullName', 'HtmlUrl', 'Description', 'Language','OwnerLogin', 'OwnerType', 'CreatedAt', 'UpdatedAt', 'PushedAt', 'License','WatchersCount']

dataset = dataset.drop(columns=columns_to_drop)

ids = dataset['Id']
dataset = dataset.drop(columns = ['Id'])



print(dataset.head())

   OpenIssuesCount  ForksCount  StargazersCount    Size  description_length
0               43       27855           331707    1525                  53
1              377       33776           316814    5030                  30
2              338       28727           306876    1058                  73
3               55       76810           306451   22318                  69
4               41       39078           296377  243999                 100


In [329]:
#normalise

from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['OpenIssuesCount', 'ForksCount', 'Size', 'description_length']
data_to_normalize = dataset[columns_to_normalize]

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data_to_normalize)

dataset[columns_to_normalize] = normalized_data

print(dataset.head())

   OpenIssuesCount  ForksCount  StargazersCount      Size  description_length
0         0.001405    0.323685           331707  0.000062                  53
1         0.012317    0.393350           316814  0.000204                  30
2         0.011043    0.333945           306876  0.000043                  73
3         0.001797    0.899674           306451  0.000905                  69
4         0.001340    0.455732           296377  0.009895                 100


In [330]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X = scaler.fit_transform(dataset[['OpenIssuesCount', 'ForksCount', 'Size', 'description_length']])
Y = dataset['StargazersCount'].values


X_train, X_temp, y_train, y_temp, ids_train, ids_temp = train_test_split( X, Y, ids, test_size=0.4, random_state=42)


X_val, X_test, y_val, y_test, ids_val, ids_test = train_test_split(X_temp, y_temp, ids_temp, test_size=0.5, random_state=42)

print(X_train[:5])

[[3.16920966e-01 2.03640300e-01 1.53213288e-02 4.62724936e-03]
 [9.18090633e-03 1.36811267e-01 7.17448832e-03 3.42759212e-03]
 [8.03737707e-03 1.09067806e-02 1.97009079e-04 1.74807198e-02]
 [5.47913876e-02 5.25219724e-02 9.55433205e-03 9.42587832e-03]
 [2.71179795e-03 2.26724554e-02 4.05129828e-05 6.34104542e-03]]


In [331]:
scaler_target = MinMaxScaler()

# Fit the scaler on the training target and transform it
y_train_scaled = scaler_target.fit_transform(y_train.reshape(-1, 1))

# Apply the same transformation to the validation set
y_val_scaled = scaler_target.transform(y_val.reshape(-1, 1))

y_test_scaled = scaler_target.transform(y_test.reshape(-1, 1))


In [332]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error  # for regression tasks
import numpy as np

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # reshape for single output
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

print(X_train[:5])

tensor([[3.1692e-01, 2.0364e-01, 1.5321e-02, 4.6272e-03],
        [9.1809e-03, 1.3681e-01, 7.1745e-03, 3.4276e-03],
        [8.0374e-03, 1.0907e-02, 1.9701e-04, 1.7481e-02],
        [5.4791e-02, 5.2522e-02, 9.5543e-03, 9.4259e-03],
        [2.7118e-03, 2.2672e-02, 4.0513e-05, 6.3410e-03]])


In [333]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 64)  # Assuming 3 input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1) 

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation for hidden layers
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [334]:
model = SimpleNN()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [335]:
def train(model, X_train, y_train, X_val, y_val, epochs):
    for epoch in range(epochs):
        model.train()
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape to column vector
        
        optimizer.zero_grad()  # Zero gradients
        predictions = model(X_train_tensor)  # Forward pass
        loss = criterion(predictions, y_train_tensor)  # Compute loss
        
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        # Validation step
        if (epoch + 1) % 100 == 0:
            model.eval()
            with torch.no_grad():
                X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
                y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
                val_predictions = model(X_val_tensor)
                val_loss = criterion(val_predictions, y_val_tensor)

            # Print losses for each epoch
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")


In [336]:
train(model, X_train, y_train_scaled, X_val, y_val_scaled, epochs=1000)

/var/folders/8r/vk9p_0jn559ggvy1pwy_gczm0000gn/T/ipykernel_32377/47673160.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
/var/folders/8r/vk9p_0jn559ggvy1pwy_gczm0000gn/T/ipykernel_32377/47673160.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_val, dtype=torch.float32)


Epoch [100/1000], Train Loss: 0.0093, Val Loss: 0.0123
Epoch [200/1000], Train Loss: 0.0084, Val Loss: 0.0114
Epoch [300/1000], Train Loss: 0.0075, Val Loss: 0.0104
Epoch [400/1000], Train Loss: 0.0067, Val Loss: 0.0093
Epoch [500/1000], Train Loss: 0.0059, Val Loss: 0.0083
Epoch [600/1000], Train Loss: 0.0054, Val Loss: 0.0075
Epoch [700/1000], Train Loss: 0.0051, Val Loss: 0.0069
Epoch [800/1000], Train Loss: 0.0050, Val Loss: 0.0067
Epoch [900/1000], Train Loss: 0.0049, Val Loss: 0.0065
Epoch [1000/1000], Train Loss: 0.0049, Val Loss: 0.0064


In [337]:
# Assuming you have test data X_test
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_tensor)
    
# Convert the predictions back to the original scale of the target variable
y_test_pred = scaler_target.inverse_transform(y_test_pred_scaled.detach().numpy())


/var/folders/8r/vk9p_0jn559ggvy1pwy_gczm0000gn/T/ipykernel_32377/813048677.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)


In [338]:
print(y_test_pred[:10])
print(y_test[:10])

[[ 33242.81 ]
 [ 26684.627]
 [ 32737.352]
 [147571.72 ]
 [ 33948.547]
 [ 25814.445]
 [ 51810.883]
 [ 29358.273]
 [ 26734.504]
 [ 30791.553]]
tensor([[21064.],
        [23680.],
        [25810.],
        [78940.],
        [45386.],
        [32945.],
        [48390.],
        [22882.],
        [22688.],
        [23864.]])


In [214]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming the model is already trained, and you've made predictions on the test set (X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_tensor)

# Inverse transform the predictions back to original scale if you normalized the target
y_test_pred = scaler_target.inverse_transform(y_test_pred_scaled.detach().numpy())

# # If you normalized your test labels as well, do the same for y_test
# y_test_original = scaler_target.inverse_transform(y_test.reshape(-1, 1))

# Calculate the metrics
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred, squared=False)  # RMSE
r2 = r2_score(y_test, y_test_pred)

# Print out the metrics
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R-squared (R²): {r2:.4f}')


Mean Absolute Error (MAE): 19661.2129
Mean Squared Error (MSE): 1831776384.0000
Root Mean Squared Error (RMSE): 42799.2578
R-squared (R²): -0.2313


/var/folders/8r/vk9p_0jn559ggvy1pwy_gczm0000gn/T/ipykernel_32377/1461250871.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
